In [1]:
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(argparse))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”


## Four Beta Model Vizualization

In [ ]:
# define command line arguments
parser <- ArgumentParser(description = "Visualize linear modeling results")
# add arguments
parser$add_argument('--celltype', type='character', help='Cell type to visualize')

# parse arguments from command line
args <- parser$parse_args()

# define cell type
celltype <- args$celltype


In [2]:

lm_file <- file.path(paste0("./results/", celltype, "/lm_four_beta.tsv"))

lm_cp_fig <- file.path(paste0("./figures/", celltype, "/lm_four_beta.pdf"))

# if path does not exist, create it
if (!dir.exists(file.path(paste0("./figures/", celltype)))) {
    dir.create(file.path(paste0("./figures/", celltype)))
}
     
lm_df <- readr::read_tsv(lm_file, col_types = readr::cols(.default = "d", feature ="c", inducer1__inducer1_dose__inhibitor__inhibitor_dose = "c"))
head(lm_df)

feature,r2_score,Metadata_number_of_singlecells,fourb_Treatment,fourb_Treatment_Dose,fourb_Inhibitor,fourb_Inhibitor_Dose,inducer1__inducer1_dose__inhibitor__inhibitor_dose
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Cytoplasm_AreaShape_Area,9.343787e-04,1.695962e-05,0.017406046,-0.017406046,0.017406046,-0.017406046,media ctr__0__Media ctr__0.0
Cytoplasm_AreaShape_BoundingBoxArea,1.740451e-03,-3.367326e-05,0.017483416,-0.017483416,0.017483416,-0.017483416,media ctr__0__Media ctr__0.0
Cytoplasm_AreaShape_BoundingBoxMaximum_X,1.077223e-04,-2.358903e-05,-0.006617265,0.006617265,-0.006617265,0.006617265,media ctr__0__Media ctr__0.0
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,1.354607e-04,-2.231822e-05,0.002458562,-0.002458562,0.002458562,-0.002458562,media ctr__0__Media ctr__0.0
Cytoplasm_AreaShape_BoundingBoxMinimum_X,1.515569e-04,-2.184418e-05,-0.007811218,0.007811218,-0.007811218,0.007811218,media ctr__0__Media ctr__0.0
Cytoplasm_AreaShape_BoundingBoxMinimum_Y,8.637752e-05,-2.176965e-05,0.001069714,-0.001069714,0.001069714,-0.001069714,media ctr__0__Media ctr__0.0


In [3]:
unique(lm_df$inducer1__inducer1_dose__inhibitor__inhibitor_dose)

[1] "media ctr__0__Media ctr__0.0"               
 [2] "DMSO__0.100__DMSO__1.0"                     
 [3] "DMSO__0.100__Z-VAD-FMK__100.0"              
 [4] "DMSO__0.100__Z-VAD-FMK__30.0"               
 [5] "DMSO__0.100__DMSO__0.025"                   
 [6] "Thapsigargin__1.000__DMSO__0.025"           
 [7] "Thapsigargin__10.000__DMSO__0.025"          
 [8] "Topotecan__5.000__DMSO__0.025"              
 [9] "Topotecan__10.000__DMSO__0.025"             
[10] "Topotecan__20.000__DMSO__0.025"             
[11] "LPS__0.010__DMSO__0.025"                    
[12] "LPS__0.100__DMSO__0.025"                    
[13] "LPS__1.000__DMSO__0.025"                    
[14] "LPS__10.000__DMSO__0.025"                   
[15] "LPS__10.000__Disulfiram__0.1"               
[16] "LPS__10.000__Disulfiram__1.0"               
[17] "LPS__10.000__Disulfiram__2.5"               
[18] "LPS_Nigericin__100.000_1.0__DMSO__0.025"    
[19] "LPS_Nigericin__100.000_3.0__DMSO__0.025"    
[20] "LPS_Nigericin__100.000_10.0__DMSO__0.025"   
[21] "Disulfiram__0.100__DMSO__0.025"             
[22] "Disulfiram__1.000__DMSO__0.025"             
[23] "Disulfiram__2.500__DMSO__0.025"             
[24] "H2O2__100.000__DMSO__0.025"                 
[25] "LPS__10.000__Z-VAD-FMK__100.0"              
[26] "LPS__100.000__DMSO__0.025"                  
[27] "LPS_Nigericin__1.000_1.0__DMSO__0.025"      
[28] "LPS_Nigericin__1.000_3.0__DMSO__0.025"      
[29] "LPS_Nigericin__1.000_10.0__DMSO__0.025"     
[30] "LPS_Nigericin__1.000_10.0__Disulfiram__1.0" 
[31] "LPS_Nigericin__1.000_10.0__Z-VAD-FMK__100.0"
[32] "H2O2__100.000__Disulfiram__1.0"             
[33] "H2O2__100.000__Z-VAD-FMK__100.0"            
[34] "Flagellin__0.100__DMSO__0.025"              
[35] "Flagellin__1.000__DMSO__0.025"              
[36] "Flagellin__1.000__Disulfiram__1.0"

In [4]:

# Arrange by absolute value coefficient
# Split out components of feature name for visualization
lm_df <- lm_df %>%
    dplyr::arrange(desc(abs(fourb_Inhibitor_Dose))) %>%
    tidyr::separate(
        feature,
        into = c(
            "compartment",
            "feature_group",
            "measurement",
            "channel",
            "parameter1",
            "parameter2"
        ),
        sep = "_",
        remove = FALSE
    ) %>%
    dplyr::mutate(channel_cleaned = channel)



Warning message:
“Expected 6 pieces. Additional pieces discarded in 28080 rows [18, 21, 22, 24, 26, 27, 28, 29, 30, 31, 33, 37, 39, 42, 43, 44, 45, 46, 47, 48,
...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 42012 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 23, ...].”


In [5]:
unique(lm_df$channel)

[1] "CorrPM"        "CorrDNA"       "CorrER"        "CorrGasdermin"
 [5] "CorrMito"      NA              "Adjacent"      "Number"       
 [9] "0"             "2"             "4"             "6"            
[13] "5"             "3"             "1"             "7"            
[17] "8"             "9"             "X"             "Y"            
[21] "Z"             "Count"

In [6]:
unique(lm_df$channel_cleaned)

[1] "CorrPM"        "CorrDNA"       "CorrER"        "CorrGasdermin"
 [5] "CorrMito"      NA              "Adjacent"      "Number"       
 [9] "0"             "2"             "4"             "6"            
[13] "5"             "3"             "1"             "7"            
[17] "8"             "9"             "X"             "Y"            
[21] "Z"             "Count"

In [7]:
# Clean channel for visualization
lm_df$channel_learned <- dplyr::recode(lm_df$channel_cleaned,
        "CorrDNA" = "nuclei",
        "CorrMito" = "Mito",
        "CorrER" = "ER",
        "CorrGasdermin" = "gasdermin",
        "CorrPM" = "PM",
        .default = "other",
        .missing="other"
    )

print(dim(lm_df))
head(lm_df)
unique(lm_df$channel_learned)
lm_df$abs_Metadata_number_of_singlecells <- abs(lm_df$Metadata_number_of_singlecells)
lm_df$abs_fourb_Treatment <- abs(lm_df$fourb_Treatment)
lm_df$abs_fourb_Treatment_Dose <- abs(lm_df$fourb_Treatment_Dose)
lm_df$abs_fourb_Inhibitor <- abs(lm_df$fourb_Inhibitor)
lm_df$abs_fourb_Inhibitor_Dose <- abs(lm_df$fourb_Inhibitor_Dose)

[1] 102492     16


feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,fourb_Treatment,fourb_Treatment_Dose,fourb_Inhibitor,fourb_Inhibitor_Dose,inducer1__inducer1_dose__inhibitor__inhibitor_dose,channel_cleaned,channel_learned
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Cytoplasm_Correlation_Manders_CorrPM_CorrMito,Cytoplasm,Correlation,Manders,CorrPM,CorrMito,NA,0.3974507,-1.330689e-05,-0.6931147,-0.6931147,-0.6931147,-0.6931147,H2O2__100.000__Disulfiram__1.0,CorrPM,PM
Cells_Correlation_Manders_CorrPM_CorrMito,Cells,Correlation,Manders,CorrPM,CorrMito,NA,0.3931833,-4.000428e-05,-0.6843945,-0.6843945,-0.6843945,-0.6843945,H2O2__100.000__Disulfiram__1.0,CorrPM,PM
Cells_Correlation_Manders_CorrDNA_CorrMito,Cells,Correlation,Manders,CorrDNA,CorrMito,NA,0.2683756,-6.649527e-05,-0.6723933,-0.6723933,-0.6723933,-0.6723933,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei
Nuclei_Correlation_Manders_CorrPM_CorrMito,Nuclei,Correlation,Manders,CorrPM,CorrMito,NA,0.1566935,2.525405e-05,-0.6616455,-0.6616455,-0.6616455,-0.6616455,H2O2__100.000__Disulfiram__1.0,CorrPM,PM
Cytoplasm_Correlation_Manders_CorrDNA_CorrMito,Cytoplasm,Correlation,Manders,CorrDNA,CorrMito,NA,0.2701912,5.697376e-06,-0.6335592,-0.6335592,-0.6335592,-0.6335592,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei
Nuclei_Correlation_Manders_CorrDNA_CorrMito,Nuclei,Correlation,Manders,CorrDNA,CorrMito,NA,0.1397890,5.213628e-05,-0.6247187,-0.6247187,-0.6247187,-0.6247187,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei


[1] "PM"        "nuclei"    "ER"        "gasdermin" "Mito"      "other"

In [8]:
head(lm_df)

feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,fourb_Treatment,⋯,fourb_Inhibitor,fourb_Inhibitor_Dose,inducer1__inducer1_dose__inhibitor__inhibitor_dose,channel_cleaned,channel_learned,abs_Metadata_number_of_singlecells,abs_fourb_Treatment,abs_fourb_Treatment_Dose,abs_fourb_Inhibitor,abs_fourb_Inhibitor_Dose
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cytoplasm_Correlation_Manders_CorrPM_CorrMito,Cytoplasm,Correlation,Manders,CorrPM,CorrMito,NA,0.3974507,-1.330689e-05,-0.6931147,⋯,-0.6931147,-0.6931147,H2O2__100.000__Disulfiram__1.0,CorrPM,PM,1.330689e-05,0.6931147,0.6931147,0.6931147,0.6931147
Cells_Correlation_Manders_CorrPM_CorrMito,Cells,Correlation,Manders,CorrPM,CorrMito,NA,0.3931833,-4.000428e-05,-0.6843945,⋯,-0.6843945,-0.6843945,H2O2__100.000__Disulfiram__1.0,CorrPM,PM,4.000428e-05,0.6843945,0.6843945,0.6843945,0.6843945
Cells_Correlation_Manders_CorrDNA_CorrMito,Cells,Correlation,Manders,CorrDNA,CorrMito,NA,0.2683756,-6.649527e-05,-0.6723933,⋯,-0.6723933,-0.6723933,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei,6.649527e-05,0.6723933,0.6723933,0.6723933,0.6723933
Nuclei_Correlation_Manders_CorrPM_CorrMito,Nuclei,Correlation,Manders,CorrPM,CorrMito,NA,0.1566935,2.525405e-05,-0.6616455,⋯,-0.6616455,-0.6616455,H2O2__100.000__Disulfiram__1.0,CorrPM,PM,2.525405e-05,0.6616455,0.6616455,0.6616455,0.6616455
Cytoplasm_Correlation_Manders_CorrDNA_CorrMito,Cytoplasm,Correlation,Manders,CorrDNA,CorrMito,NA,0.2701912,5.697376e-06,-0.6335592,⋯,-0.6335592,-0.6335592,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei,5.697376e-06,0.6335592,0.6335592,0.6335592,0.6335592
Nuclei_Correlation_Manders_CorrDNA_CorrMito,Nuclei,Correlation,Manders,CorrDNA,CorrMito,NA,0.1397890,5.213628e-05,-0.6247187,⋯,-0.6247187,-0.6247187,H2O2__100.000__Disulfiram__1.0,CorrDNA,nuclei,5.213628e-05,0.6247187,0.6247187,0.6247187,0.6247187


In [9]:
loop_list <- unique(lm_df$inducer1__inducer1_dose__inhibitor__inhibitor_dose)
x_list <- c('abs_fourb_Treatment','abs_fourb_Treatment_Dose','abs_fourb_Inhibitor','abs_fourb_Inhibitor_Dose')

In [10]:
pdf(file=lm_cp_fig )
for (i in 1:length(loop_list)){
    df <- lm_df[lm_df$inducer1__inducer1_dose__inhibitor__inhibitor_dose == loop_list[i],]
    for (j in 1:length(x_list)){
        lm_fig_gg <- (
            ggplot(df, aes(x = .data[[x_list[j]]], y = r2_score))
            + geom_point(aes(size = abs_Metadata_number_of_singlecells, color = channel_learned), alpha = 0.7)
            + theme_bw()
            + guides(
                color = guide_legend(title = "Channel\n(if applicable)", order = 1),
                size = guide_legend(title = "Cell count contributution")
            )
            + ylab("R2 score of LM feature")
            + xlab(paste0(x_list[j]," contribution (LM beta coefficient)"))
            + ggtitle(paste0("How CellProfiler features contribute\nto ",loop_list[i], "\ntreatments and cell density"))
        )
    plot(lm_fig_gg)    
    }
}
dev.off()

png 
  2